In [2]:
import pandas  as pd

def read_data(fname, num_sent=4):
    contexts = []
    endings = []
    labels = []
    data = pd.read_csv(fname)
    for idx, row in data.iterrows():
        sents = []
        for i in [4,3,2,1]:
            if len(sents) == num_sent:
                break
            sents.insert(0, row[f'Kalimat-{i}'])
        context = ' '.join(sents) # row['Kalimat-1'] +' '+ row['Kalimat-2'] +' '+ row['Kalimat-3'] +' '+ row['Kalimat-4']
        ending1 = row['Correct Ending']
        ending2 = row['Incorrect Ending']
        
        contexts.append(context)
        endings.append(ending1)
        labels.append(1)
        
        contexts.append(context)
        endings.append(ending2)
        labels.append(0)
    return contexts, endings, labels

In [4]:
import io

def load_vectors(fname, word_index):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        if word_index.get(tokens[0],-1) != -1:
            data[tokens[0]] = list(map(float, tokens[1:]))
    return data

contexts, endings, _ = read_data('Data/data_id/test.csv', 4)
all_sentences = contexts + endings
word_index = {}

for sentence in all_sentences:
    for word in sentence.split():
        word_index[word.lower()] = 1

vecs = load_vectors('cc.id.300.vec', word_index)

In [10]:
from numpy import dot
from numpy.linalg import norm
import numpy as np


def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

def get_score(sent1, sent2):
    arr1 = np.zeros(300)
    for word in sent1.split():
        if vecs.get(word.lower(), 0) != 0:
            arr1+=vecs.get(word.lower())
    
    arr2 = np.zeros(300)
    for word in sent2.split():
        if vecs.get(word.lower(), 0) != 0:
            arr2+=vecs.get(word.lower())
    arr1 = arr1 / len(sent1.split())       
    arr2 = arr2 / len(sent2.split())
    return cos_sim(arr1, arr2)

In [11]:
for num_sent in [1,2,3,4]:
    scores = []; true = 0; idy = 0
    contexts, endings, _ = read_data('Data/data_id/test.csv', num_sent)
    for idx, context in enumerate(contexts):
        scores.append(get_score(context, endings[idx]))
    while idy < len(scores):
        if scores[idy] > scores[idy+1]:
            true+=1        
        idy+=2
    print(num_sent, true/(len(contexts)/2))

1 0.5894273127753304
2 0.6229074889867842
3 0.6246696035242291
4 0.6255506607929515
